# [AHA! Activity Health Analytics](http://casas.wsu.edu/)
[Center for Advanced Studies of Adaptive Systems (CASAS)](http://casas.wsu.edu/)

[Washington State University](https://wsu.edu)

# L2 Fitbit Activity Data

## Learner Objectives
At the conclusion of this lesson, participants should have an understanding of:
* Interfacing with the Fitbit API
* Downloading data from the Fitbit API

## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Fitbit API docs](https://dev.fitbit.com/docs/)

## Time Series Activity Data
Let's dive into downloading the minute by minute time series data for the following intraday activity resources (from the [Fitbit activities docs](https://dev.fitbit.com/docs/activity/#get-activity-intraday-time-series):
* activities/calories  
* activities/steps  
* activities/distance  
* activities/floors  
* activities/elevation
* activities/heart

Note: not all Fitbit models have the above information available. I will be downloading data from my Fitbit Charge HR which does have all of the above resources. At a minimum, your model should at least have calories, steps, and distance.

First, we need to run our code from the previous lesson to authenticate ourselves with our Fitbit app.

In [1]:
import fitbit
import files.gather_keys_oauth2 as gather

# storing Fitbit app info external to this program
in_file = open(r"files\fitbit_client_keys.txt", "r")
app_info = in_file.readlines()
CLIENT_ID = app_info[0].strip()
CLIENT_SECRET = app_info[1].strip()
server = gather.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
USER_ID = server.fitbit.client.session.token['user_id']
ACCESS_TOKEN =  server.fitbit.client.session.token['access_token']
REFRESH_TOKEN =  server.fitbit.client.session.token['refresh_token']

authd_client = fitbit.Fitbit(USER_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)
user_stats = authd_client.user_profile_get('-')
display_name = user_stats["user"]["displayName"]
print(display_name)

[07/Jun/2017:13:52:22] ENGINE Listening for SIGTERM.
[07/Jun/2017:13:52:22] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[07/Jun/2017:13:52:22] ENGINE Set handler for console events.
[07/Jun/2017:13:52:22] ENGINE Started monitor thread 'Autoreloader'.
[07/Jun/2017:13:52:22] ENGINE Started monitor thread '_TimeoutMonitor'.
[07/Jun/2017:13:52:22] ENGINE Serving on http://127.0.0.1:8080
[07/Jun/2017:13:52:22] ENGINE Bus STARTED


127.0.0.1 - - [07/Jun/2017:13:52:35] "GET /?code=e661dfb8c6f00ed2680bbaa4c3c233032e51503a&state=WG9N5thbtTgWo5vjkSLGBu6NLARKC5 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
127.0.0.1 - - [07/Jun/2017:13:52:36] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/?code=e661dfb8c6f00ed2680bbaa4c3c233032e51503a&state=WG9N5thbtTgWo5vjkSLGBu6NLARKC5" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"


[07/Jun/2017:13:52:36] ENGINE Bus STOPPING
[07/Jun/2017:13:52:46] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[07/Jun/2017:13:52:46] ENGINE Stopped thread 'Autoreloader'.
[07/Jun/2017:13:52:46] ENGINE Stopped thread '_TimeoutMonitor'.
[07/Jun/2017:13:52:46] ENGINE Removed handler for console events.
[07/Jun/2017:13:52:46] ENGINE Bus STOPPED
[07/Jun/2017:13:52:46] ENGINE Bus EXITING
[07/Jun/2017:13:52:46] ENGINE Waiting for child threads to terminate...
[07/Jun/2017:13:52:46] ENGINE Bus EXITED
[07/Jun/2017:13:52:46] ENGINE Waiting for thread Thread-20.


Gina


### Steps -> Series
We will go through an example of querying and extracting intraday data for step counts. The example day I am going to use February 23rd, 2017, a day I walked 12,579 steps.
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/fitbit_2-23-17.png" width="600">

In [2]:
# year-month-day
date = "2017-02-23"
# base_data default is "today"
# other parameters include detail_level, start_time and end_time
# detail_level default is "1min"
# can also be "15min" (optional). "1sec" for heart rate.
intraday_steps = authd_client.intraday_time_series('activities/steps', base_date=date)
steps_data = intraday_steps["activities-steps-intraday"]["dataset"]

Let's now extract each minute time stamp and its associated number of steps from the `steps_data` list. We will store the time series data in a Pandas `Series` object with timestamps (string) as the index and step counts (integer) as the values.

In [3]:
import pandas as pd

def extract_intraday_series(data, time_label="time"):
    '''
    
    '''
    timestamps = []
    values = []
    for datapoint in data:
        timestamps.append(datapoint[time_label])
        values.append(datapoint["value"])

    ser = pd.Series(values, index=timestamps)
    return ser

steps_ser = extract_intraday_series(steps_data)
print("Total steps taken on", date, ":", steps_ser.sum())

Total steps taken on 2017-02-23 : 12579


### Activity Series -> DataFrame
Now that we can extract a series for steps, like extract all activity series and stack them to form a data frame. 

In [4]:
def collect_intraday_activity_data(authd_client, date, activities):
    '''
    
    '''
    activity_dict = {}
    for act in activities:
        res = "activities/" + act
        intraday = authd_client.intraday_time_series(res, base_date=date)
        intraday_data = intraday["activities-" + act + "-intraday"]["dataset"]
        ser = extract_intraday_series(intraday_data)
        activity_dict[act] = ser
    df = pd.DataFrame(activity_dict)
    return df
        
activities = ["calories", "steps", "distance", "floors", "elevation", "heart"]
df = collect_intraday_activity_data(authd_client, date, activities)

In [5]:
print(df.head(), end="\n\n")
print(df.describe())
out_fname = "files\\" + display_name + "_" + date + "_df.csv"
df.to_csv(out_fname)

          calories  distance  elevation  floors  heart  steps
00:00:00    0.9652       0.0          0       0   62.0      0
00:01:00    0.9652       0.0          0       0   64.0      0
00:02:00    0.9652       0.0          0       0   58.0      0
00:03:00    0.9652       0.0          0       0   60.0      0
00:04:00    0.9652       0.0          0       0   61.0      0

          calories     distance    elevation       floors        heart  \
count  1440.000000  1440.000000  1440.000000  1440.000000  1336.000000   
mean      1.537282     0.003806     0.388889     0.038889    70.965569   
std       1.264977     0.011068     2.837041     0.283704    12.412764   
min       0.965200     0.000000     0.000000     0.000000    53.000000   
25%       0.965200     0.000000     0.000000     0.000000          NaN   
50%       0.965200     0.000000     0.000000     0.000000          NaN   
75%       1.061720     0.000000     0.000000     0.000000          NaN   
max       8.300720     0.053624    

C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Note: If the device is unable to get a heart rate reading then the heart rate data is null. 

## API Rate Limit
Your application can make 150 API requests per hour without a user access token. These types of API requests are for retrieving non-user data, such as Fitbit's general resources (Browse Activities, Get Activity, Search Foods, Get Food, and Get Food Units).